In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from urllib.parse import quote
import time

In [ ]:
city = ['manchester']

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
            'Accept-Language': 'en-US, en;q=0.5'})

f_name = 'main_house_links_' + city[0] + '.xlsx'
city_houses = pd.read_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'links', f_name))

links = list(city_houses['houses'])

reg = re.compile(r'[0-9]+')
# reg_map = re.compile(r'&center=(.*)&maptype')

check = 0
errored = []
cols = ['id', 'bed', 'bath', 'reception', 'terrace', 'house', 'flat', 'bungalow', 'tenure', 'lat', 'lon', 'ch', 'dg', 'price']
api_key = 'pk.1a34e455dc7a1c2ce1b1b7c1da0877e7'

In [ ]:
data = pd.DataFrame(columns=cols)
for link in links:

    if check < 0:
        check += 1
        continue

    time.sleep(1)

    if check % 100 == 0:
        if len(data) > 0:
            # saving data in small batches
            f_name = str(check) + '.xlsx'
            data.to_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'data', city[0], f_name))
            
            # writing errored links
            f_name = str(check) + '.txt'
            with open(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'data', city[0], f_name), 'w') as fp:
                for item in errored:
                    # write each item on a new line
                    fp.write("%s\n" % item)

            time.sleep(15)
            print(check)
            data = pd.DataFrame(columns=cols)
            errored.clear()
            check += 1
        else:
            check += 1
    else:
        check += 1

    data_dict = {a:b for a, b in zip(cols, [0 for _ in range(len(cols))])} 

    data_dict['id'] = re.findall(reg, link)[0]

    try:
        page = requests.get(link, headers=HEADERS )
    except:
        print("Can't find -> ", link)
        continue
    
    print(check)

    try:
        soup = BeautifulSoup( page.content , 'html.parser')

        # bed, bath, reception, terrace, house, flat, bunglow, tenure, lat, lon, ch, dg, price = [0 for _ in range(13)]
        
        # getting data about beds, baths and reception
        # c-PJLV c-PJLV-kQvhQW-centered-true c-PJLV-iPJLV-css
        for i in soup.find_all('div', {'class' : 'c-PJLV c-PJLV-kQvhQW-centered-true c-PJLV-iPJLV-css'}):
            temp = i.text
            if 'bed' in temp:
                data_dict['bed'] = int(re.findall(reg,temp)[0])
            elif 'bath' in temp:
                data_dict['bath'] = int(re.findall(reg,temp)[0])
            elif 'reception' in temp:
                data_dict['reception'] = int(re.findall(reg,temp)[0])

        # getting info about terrace and house type
        h = soup.find_all('h1', {'id' : 'listing-summary-details-heading'})[0].text
        h = h.lower()

        if 'terrace' in h:
            data_dict['terrace'] = 1
        if 'house' in h:
            data_dict['house'] = 1
        if 'flat' in h:
            data_dict['flat'] = 1
        if 'bungalow' in h:
            data_dict['bungalow'] = 1
        
        if [data_dict['house'], data_dict['flat'], data_dict['bungalow']] == [0,0,0]:
            print(h)

        # getting lat lng
        t = soup.find_all('address')
        
        # using geopy
        # temp  = geolocator.geocode(t[0].text)
        # data_dict['lat'] = temp.latitude
        # data_dict['lon'] = temp.longitude

        # using api
        # print(t[0].text)
        # change api key
        x = f'https://eu1.locationiq.com/v1/search?key={api_key}&q={quote(t[0].text)}&format=json'
        y = requests.get(x)

        data_dict['lat'] = y.json()[0]['lat']
        data_dict['lon'] = y.json()[0]['lon']


        # getting tenure
        # c-PJLV c-PJLV-ijgVXxo-css
        t = soup.find_all('div', {'class' : 'c-PJLV c-PJLV-ijgVXxo-css'})[0].text.split(":")
        
        if 'Tenure' == t[0]:
            data_dict['tenure'] = t[1]

        # getting info about central heating and double glazing
        t = soup.find('ul', {'data-testid' : 'listing_features_bulletted'})
        if not t:
            t = soup.find('div', {'data-testid' : 'listing_description'})
            if t:
              t = t.text
              t = t.lower()

              if ('central' in t) and ('heat' in t):
                      data_dict['ch'] = 1
              if ('double' in t) and ('glaz' in t):
                  data_dict['dg'] = 1
        else:
            t = [i.text for i in t.find_all('li')]

            for i in t:
                i = i.lower()
                if ('central' in i) and ('heat' in i):
                    data_dict['ch'] = 1
                if ('double' in i) and ('glaz' in i):
                    data_dict['dg'] = 1

            if data_dict['ch'] == 0 and data_dict['dg'] == 0:
                t = soup.find('div', {'data-testid' : 'listing_description'})
                if t:
                  t = t.text
                  t = t.lower()

                  if ('central' in t) and ('heat' in t):
                          data_dict['ch'] = 1
                  if ('double' in t) and ('glaz' in t):
                      data_dict['dg'] = 1


        # getting price
        t = soup.find('p', {'data-testid' : 'price'})
        data_dict['price'] = ''.join(re.findall(reg,t.text))

        data = data.append(data_dict, ignore_index = True)
        # print(data)

    except Exception as e:
        print("Can't get ", link)
        errored.append(link)
        print(e)

if len(data) > 0:
  f_name = str(check) + '.xlsx'
  data.to_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'data', city[0], f_name))
  # print(data)

1
2
3
4
5
6
7
8
Can't get  https://www.zoopla.co.uk/new-homes/details/62656026
list index out of range
9
Can't get  https://www.zoopla.co.uk/new-homes/details/62656027
list index out of range
10
Can't get  https://www.zoopla.co.uk/new-homes/details/62656028
list index out of range
11
Can't get  https://www.zoopla.co.uk/new-homes/details/62656029
list index out of range
12
13
14
15
16
17
18
19
Can't get  https://www.zoopla.co.uk/new-homes/details/62654556
list index out of range
20
Can't get  https://www.zoopla.co.uk/new-homes/details/62654557
list index out of range
21
Can't get  https://www.zoopla.co.uk/new-homes/details/62654550
list index out of range
22
Can't get  https://www.zoopla.co.uk/new-homes/details/62654551
list index out of range
23
Can't get  https://www.zoopla.co.uk/new-homes/details/62654529
list index out of range
24
Can't get  https://www.zoopla.co.uk/new-homes/details/62654527
list index out of range
25
Can't get  https://www.zoopla.co.uk/new-homes/details/62654520
l

## FOR TESTING

In [ ]:
check = 0
# https://www.zoopla.co.uk/new-homes/details/62440471/
# https://www.zoopla.co.uk/new-homes/details/62440470/
# https://www.zoopla.co.uk/for-sale/details/62647568/
# https://www.zoopla.co.uk/new-homes/details/61491792/
# https://www.zoopla.co.uk/new-homes/details/62613924/
# https://www.zoopla.co.uk/for-sale/details/62588779/
# https://www.zoopla.co.uk/for-sale/details/62651446/
links = ['https://www.zoopla.co.uk/for-sale/details/62588779/']
data = pd.DataFrame(columns=cols)
for link in links:
    time.sleep(1)
    print(1)
    if check < 0:
        continue
        check += 1

    if check % 100 == 0:
        if len(data) > 0:
            # saving data in small batches
            f_name = str(check) + '.xlsx'
            data.to_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'data', city[0], f_name))
            
            # writing errored links
            f_name = str(check) + '.txt'
            with open(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'data', city[0], f_name), 'w') as fp:
                for item in errored:
                    # write each item on a new line
                    fp.write("%s\n" % item)

            time.sleep(15)
            print(check)
            data = pd.DataFrame(columns=cols)
            errored.clear()
            check += 1
        elif check != 0:
            print("NO DATA COMING..................")
            break
        else:
            check += 1
    else:
        check += 1

    print(2)
    data_dict = {a:b for a, b in zip(cols, [0 for _ in range(len(cols))])} 

    data_dict['id'] = re.findall(reg, link)[0]

    try:
        page = requests.get(link, headers=HEADERS )
    except:
        print("Can't find -> ", link)
        continue
    
    print(check)

    soup = BeautifulSoup( page.content , 'html.parser')

    # bed, bath, reception, terrace, house, flat, bunglow, tenure, lat, lon, ch, dg, price = [0 for _ in range(13)]
    
    # getting data about beds, baths and reception
    # c-PJLV c-PJLV-kQvhQW-centered-true c-PJLV-iPJLV-css
    for i in soup.find_all('div', {'class' : 'c-PJLV c-PJLV-kQvhQW-centered-true c-PJLV-iPJLV-css'}):
        temp = i.text
        if 'bed' in temp:
            data_dict['bed'] = int(re.findall(reg,temp)[0])
        elif 'bath' in temp:
            data_dict['bath'] = int(re.findall(reg,temp)[0])
        elif 'reception' in temp:
            data_dict['reception'] = int(re.findall(reg,temp)[0])

    # getting info about terrace and house type
    h = soup.find_all('h1', {'id' : 'listing-summary-details-heading'})[0].text

    if 'terrace' in h:
        data_dict['terrace'] = 1
    if 'house' in h:
        data_dict['house'] = 1
    if 'flat' in h:
        data_dict['flat'] = 1
    if 'bungalow' in h:
        data_dict['bungalow'] = 1
    
    if [data_dict['house'], data_dict['flat'], data_dict['bungalow']] == [0,0,0]:
        print(h)

    # getting lat lng
    t = soup.find_all('address')
    
    # using geopy
    # temp  = geolocator.geocode(t[0].text)
    # data_dict['lat'] = temp.latitude
    # data_dict['lon'] = temp.longitude

    # using api
    # print(t[0].text)
    # change api key
    x = f'https://eu1.locationiq.com/v1/search?key={api_key}&q={quote(t[0].text)}&format=json'
    y = requests.get(x)
    print(y.json())
    data_dict['lat'] = y.json()[0]['lat']
    data_dict['lon'] = y.json()[0]['lon']


    # getting tenure
    # c-PJLV c-PJLV-ijgVXxo-css
    t = soup.find_all('div', {'class' : 'c-PJLV c-PJLV-ijgVXxo-css'})[0].text.split(":")
    
    if 'Tenure' == t[0]:
        data_dict['tenure'] = t[1]

    # getting info about central heating and double glazing
    t = soup.find('ul', {'data-testid' : 'listing_features_bulletted'})
    if not t:
        t = soup.find('div', {'data-testid' : 'listing_description'})
        if t:
          t = t.text
          t = t.lower()

          if ('central' in t) and ('heat' in t):
                  data_dict['ch'] = 1
          if ('double' in t) and ('glaz' in t):
              data_dict['dg'] = 1
    else:
        t = [i.text for i in t.find_all('li')]

        for i in t:
            i = i.lower()
            if ('central' in i) and ('heat' in i):
                data_dict['ch'] = 1
            if ('double' in i) and ('glaz' in i):
                data_dict['dg'] = 1

        if data_dict['ch'] == 0 and data_dict['dg'] == 0:
            t = soup.find('div', {'data-testid' : 'listing_description'})
            
            if t:
              t = t.text
              t = t.lower()

              if ('central' in t) and ('heat' in t):
                      data_dict['ch'] = 1
              if ('double' in t) and ('glaz' in t):
                  data_dict['dg'] = 1


    # getting price
    t = soup.find('p', {'data-testid' : 'price'})
    data_dict['price'] = ''.join(re.findall(reg,t.text))

    data = data.append(data_dict, ignore_index = True)
    print(data)
    print("HELO")


1
2
1
{'error': 'Unable to geocode'}


KeyError: ignored

In [ ]:
city_houses = pd.read_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'links', 'main_house_links_man_aberd.xlsx'))

city_houses_man = city_houses[city_houses['city'] == 'aberdeenshire']

In [ ]:
city_houses_man.columns

Index(['Unnamed: 0', 'houses', 'city'], dtype='object')

In [ ]:
city_houses_man.drop('Unnamed: 0', inplace = True, axis = 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
city_houses_man

,houses,city
0,https://www.zoopla.co.uk/for-sale/details/6265...,manchester
1,https://www.zoopla.co.uk/for-sale/details/6265...,manchester
2,https://www.zoopla.co.uk/for-sale/details/6265...,manchester
3,https://www.zoopla.co.uk/for-sale/details/6265...,manchester
4,https://www.zoopla.co.uk/for-sale/details/6265...,manchester
...,...,...
1020,https://www.zoopla.co.uk/for-sale/details/6243...,manchester
1021,https://www.zoopla.co.uk/for-sale/details/6243...,manchester
1022,https://www.zoopla.co.uk/new-homes/details/624...,manchester
1023,https://www.zoopla.co.uk/new-homes/details/624...,manchester


In [ ]:
city_houses_man.to_excel(os.path.join(os.getcwd(), 'drive','MyDrive', 'PC503_project', 'links', 'main_house_links_aberdeenshire.xlsx'), index = False)